## Run your application

With your data loaded and embedded, you are now ready to run your application. This application is built using Streamlit. To explore the application code, open `application.py`.
 
When you are ready, run this notebook to start the application. Switch to the *application tab name tbd* to begin using your chatbot.

In [1]:
import os

os.environ['STREAMLIT_GENERAL_EMAIL'] = ""
os.environ['STREAMLIT_THEME_PRIMARYCOLOR'] = "#5863ff"


In [ ]:
%%bash
# If you're using a container, mac, or linux machine --- run this cell
streamlit run application.py --server.port 8080

In [2]:
%%cmd
@REM If you're using a windows machine, run this cell
streamlit run application.py --server.port 8080

Microsoft Windows [Version 10.0.22000.2538]
(c) Microsoft Corporation. All rights reserved.

c:\Users\nmitchko\Documents\Sales\Events\2024\gs24-ai-workshop\exercise\healthcare>@REM If you're using a windows machine, run this cell

c:\Users\nmitchko\Documents\Sales\Events\2024\gs24-ai-workshop\exercise\healthcare>streamlit run application.py --server.port 8080

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8080
  Network URL: http://10.0.0.205:8080



C:\Users\nmitchko\AppData\Roaming\Python\Python311\site-packages\langchain\_api\module_import.py:87: LangChainDeprecationWarning: Importing get_openai_callback from C:\Users\nmitchko\AppData\Roaming\Python\Python311\site-packages\langchain\callbacks\__init__.py is deprecated. Please replace the import with the following:
from langchain_community.callbacks.manager import get_openai_callback
  warnings.warn(
C:\Users\nmitchko\AppData\Roaming\Python\Python311\site-packages\langchain\embeddings\__init__.py:29: LangChainDeprecationWarning: Importing embeddings from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.embeddings import OpenAIEmbeddings`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
C:\Users\nmitchko\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWar

No Dataset selected


C:\Users\nmitchko\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 
                    Prompt: what can you tell me about diabetes?

                    Relevant Documents: None

                    You should only make use of the provided Relevant Documents. They are important information belonging to the user, and it is important that any advice you give is grounded in these documents. If the documents are irrelevant to the question, simply state that you do not have the relevant information available in the database.
                
AI:

> Finished chain.


C:\Users\nmitchko\AppData\Roaming\Python\Python311\site-packages\langchain\_api\module_import.py:87: LangChainDeprecationWarning: Importing get_openai_callback from C:\Users\nmitchko\AppData\Roaming\Python\Python311\site-packages\langchain\callbacks\__init__.py is deprecated. Please replace the import with the following:
from langchain_community.callbacks.manager import get_openai_callback
  warnings.warn(
C:\Users\nmitchko\AppData\Roaming\Python\Python311\site-packages\langchain\embeddings\__init__.py:29: LangChainDeprecationWarning: Importing embeddings from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.embeddings import OpenAIEmbeddings`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 
                    Prompt: okay, now tell me about some diabetic patients

                    Relevant Documents: [(Document(page_content='A 78-year-old woman was referred to our department due to a mass in the lower abdomen. She had a history of type 2 diabetes mellitus and had been insulin-dependent for more than 7 years. She was admitted to the Internal Medicine Department because of deteriorating blood glucose control. Her HbA1c was 10.6% and total daily insulin dose was 58 units. On admission, physical examination revealed a firm mass in the lower abdomen with no pain or tenderness, and she was referred to our department for further treatment.\nComputed

C:\Users\nmitchko\AppData\Roaming\Python\Python311\site-packages\langchain\_api\module_import.py:87: LangChainDeprecationWarning: Importing get_openai_callback from C:\Users\nmitchko\AppData\Roaming\Python\Python311\site-packages\langchain\callbacks\__init__.py is deprecated. Please replace the import with the following:
from langchain_community.callbacks.manager import get_openai_callback
  warnings.warn(
C:\Users\nmitchko\AppData\Roaming\Python\Python311\site-packages\langchain\embeddings\__init__.py:29: LangChainDeprecationWarning: Importing embeddings from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.embeddings import OpenAIEmbeddings`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
